Analyze data for Major League Baseball gaves from Retrosheet.

Read the data dictionary provided by Retrosheet.

In [110]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


Import libraries

In [111]:
import pandas as pd
import numpy as np
import csv
import sqlite3

setting options to prevent output from being truncated

In [112]:
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [113]:
game_log=pd.read_csv('game_log.csv', low_memory=False)
print(game_log.shape)

(171907, 161)


In [114]:
print(game_log.head())
print(game_log.tail())

       date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit protest park_id

In [115]:
park_codes=pd.read_csv('park_codes.csv', low_memory=False)
print(park_codes.shape)

(252, 9)


In [116]:
print(park_codes.head())
print(park_codes.tail())

  park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                             

In [117]:
person_codes=pd.read_csv('person_codes.csv', low_memory=False)
print(person_codes.shape)

(20494, 7)


In [118]:
print(person_codes.head())
print(person_codes.tail())

         id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN
             id      last   first player_debut mgr_debut coach_debut ump_debut
20489  zuvep001   Zuvella    Paul   09/04/1982       NaN  04/02/1996       NaN
20490  zuveg101  Zuverink  George   04/21/1951       NaN         NaN       NaN
20491  zwild101  Zwilling   Dutch   08/14/1910       NaN  04/15/1941       NaN
20492  zycht001      Zych    Tony   09/04/2015       NaN         NaN       NaN
20493  thoma102  Thompson     NaN          NaN       NaN         NaN       NaN


In [119]:
team_codes=pd.read_csv('team_codes.csv', low_memory=False)
print(team_codes.shape)

(150, 8)


In [120]:
print(team_codes.head())
print(team_codes.tail())

  team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1
    team_id league  start   end        city   nickname franch_id  seq
145     WS8     NL   1886  1889  Washington   Senators       WS8    1
146     WS9     AA   1891  1891  Washington   Senators       WS9    1
147     WSN     NL   1892  1899  Washington   Senators       WS9    2
148     WSU     UA   1884  1884  Washington  Nationals       WSU    1
149     MIA     NL   2012     0       Miami    Marlins       FLO    2


The game_log file consists of identifiers such as: 
-  game_log(h/v_player_#_id) references person_codes(id)
-  game_log(h/v_name) references team_codes(team_id)
-  game_log(park_id) references park_codes(park_id)

to join the tables

build some helper functions: 

In [121]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn: 
        return pd.read_sql(q, conn)
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.isolation_level = None
        conn.execute(c)
def show_tables():
    q_show_tables='''
    SELECT
        name,
        type
    FROM 
        sqlite_master
    WHERE 
        type IN ("table","view");
    '''
    return run_query(q_show_tables)

create tables: 

In [122]:
tables={
    "game_log": game_log, 
    "team_codes": team_codes, 
    "person_codes": person_codes, 
    "park_codes": park_codes
} 
with sqlite3.connect('mlb.db') as conn:
    for sql_table_name, data in tables.items():
        conn.execute("DROP TABLE IF EXISTS {};".format(sql_table_name))
        data.to_sql(sql_table_name, conn, index=False)
        
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,position,table
5,team,table
6,game,table
7,team_performance,table
8,personal_performance,table
9,person_codes,table


add game_id column to game_log table

In [123]:
c_add_column='''
    alter table game_log
    add column game_id TEXT;
    '''
try:
    run_command(c_add_column)
except:
    pass

In [124]:
c_fill_game_id='''
    update game_log
    set game_id = v_name || date || number_of_game
    WHERE 
        game_id IS NULL;
    '''
run_command(c_fill_game_id)

opportunities to normalize the data:
-  start and end date of the park can be derived using game_log
-  start and end date of the team can be derived using game_log
-  debut date of the person can be derived using game_log
-  name of player in game_log can be dervied using person_codes
-  players' performance can be put into a different table

![Schema](mlb-schema.png)

In [125]:
c_create_person_table='''
    create table if not exists person
    (
    person_id TEXT primary key,
    first_name TEXT, 
    last_name TEXT
    )
    '''
c_insert_person_table='''
    insert or ignore into person
    select
        id, 
        first, 
        last
    from
        person_codes
    '''
q='''
    select
        *
    from
        person
    limit 5
    '''
run_command(c_create_person_table)
run_command(c_insert_person_table)
run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [126]:
c_create_park_table='''
    create table if not exists park
    (
    park_id TEXT primary key, 
    name TEXT, 
    nickname TEXT, 
    city TEXT, 
    state TEXT, 
    notes TEXT
    )
    '''
c_insert_park_table='''
    insert or ignore into park
    select
        park_id, 
        name, 
        aka, 
        city, 
        state, 
        notes
    from 
        park_codes
    '''
q='''
    select
        *
    from 
        park_codes
    limit 5
    '''
run_command(c_create_park_table)
run_command(c_insert_park_table)
run_query(q)

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,None,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,04/30/1884,05/31/1884,UA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,None,AL,None
3,ARL01,Arlington Stadium,None,Arlington,TX,04/21/1972,10/03/1993,AL,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,None,AL,None


In [127]:
c_create_league_table='''
    create table if not exists league 
    (
    league_id TEXT primary key, 
    name TEXT
    )
    '''
c_insert_league_table='''
    insert or ignore into league
    values
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association")
    '''
q='''
    select
        *
    from
        league
    limit 5
    '''
run_command(c_create_league_table)
run_command(c_insert_league_table)
run_query(q)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League


In [128]:
c_create_position_table='''
    create table if not exists position 
    (
    position_id TEXT primary key, 
    name TEXT, 
    category TEXT
    )
    '''
appearance_type = pd.read_csv('appearance_type.csv')
with sqlite3.connect('mlb.db') as conn:
    appearance_type.to_sql('appearance_type', conn, index=False, if_exists='append')
c_insert_position_table='''
    insert or ignore into position
    select
        appearance_type_id, 
        name, 
        category
    from 
        appearance_type
    '''
q='''
    select
        *
    from
        position
    '''
run_command(c_create_position_table)
run_command(c_insert_position_table)
run_query(q)

,position_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [129]:
c_create_team_table='''
    create table if not exists team 
    (
    team_id TEXT primary key, 
    league_id TEXT, 
    city TEXT, 
    nickname TEXT, 
    franch_id TEXT, 
    foreign key (league_id) references league(league_id)
    )
    '''
c_insert_team_table='''
    insert or ignore into team
    select
        team_id, 
        league, 
        city, 
        nickname, 
        franch_id
    from
        team_codes
    '''
q='''
    select
        *
    from
        team
    limit 5
    '''
run_command(c_create_team_table)
run_command(c_insert_team_table)
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [130]:
c_create_game_table='''
    create table if not exists game
    (
    game_id TEXT primary key, 
    date TEXT, 
    number_of_game INTEGER, 
    park_id TEXT, 
    length_outs INTEGER, 
    day BOOLEAN, 
    completion TEXT, 
    forefeit TEXT, 
    protest TEXT, 
    attendance INTEGER, 
    length_minutes INTEGER, 
    additional_info TEXT, 
    acquisition_info TEXT, 
    foreign key (park_id) references park(park_id)
    )
    '''
c_insert_game_table='''
    insert or ignore into game
    select
        game_id, 
        date, 
        number_of_game, 
        park_id, 
        length_outs, 
        case
            when day_night = "D" then 1
            when day_night = "N" then 0
            else null
            end day, 
        completion, 
        forefeit, 
        protest, 
        attendance, 
        length_minutes, 
        additional_info, 
        acquisition_info
    from
        game_log
    '''
q='''
    select
        *
    from
        game
    limit 5
    '''
run_command(c_create_game_table)
run_command(c_insert_game_table)
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,CL1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,BS1187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,CL1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,CL1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,BS1187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


In [131]:
c_create_team_performance_table='''
    create table if not exists team_performance
    (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    primary key (team_id, game_id), 
    foreign key (team_id) references team(team_id),
    foreign key (game_id) references game(game_id)
    )
    '''
c_insert_team_performance_table='''
    insert or ignore into team_performance
    select
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    from
        game_log
    union
    select
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from 
        game_log
    '''
q='''
    select
        *
    from
        team_performance
    where
        game_id=(
            select
                min(game_id)
            from
                game
            ) or
        game_id=(
            select
                max(game_id)
            from
                game
            )
    order by game_id, home
    '''
run_command(c_create_team_performance_table)
run_command(c_insert_team_performance_table)
run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404170,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,CNU,ALT188404170,1,UA,7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,WSU,WSU188410190,0,UA,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,KCU,WSU188410190,1,UA,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [138]:
c_create_personal_performance_table='''
    create table if not exists personal_performance
    (
    performance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    position_id,
    foreign key (person_id) references person(person_id), 
    foreign key (team_id) references team(team_id), 
    foreign key (game_id) references game(game_id), 
    foreign key (position_id) references position(position_id)
    )
    '''
c_insert_personal_performance_table='''
    insert or ignore into personal_performance
    (
    game_id, 
    team_id, 
    person_id, 
    position_id
    )
    select
        game_id, 
        null, 
        hp_umpire_id, 
        "UHP"
    from
        game_log
    where 
        hp_umpire_id is not null
    union
    select
        game_id, 
        null, 
        [1b_umpire_id], 
        "U1B"
    from 
        game_log
    where
        "1b_umpire_id" is not null
    union
    select
        game_id, 
        null, 
        [2b_umpire_id], 
        "U2B"
    from
        game_log
    where
        [2b_umpire_id] is not null
    union
    select
        game_id, 
        null, 
        [3b_umpire_id], 
        "U3B"
    from
        game_log
    where
        [3b_umpire_id] is not null
    union
    select
        game_id, 
        null, 
        [lf_umpire_id], 
        "ULF"
    from
        game_log
    where
        [lf_umpire_id] is not null
    union
    select
        game_id, 
        null, 
        [rf_umpire_id], 
        "URF"
    from
        game_log
    where
        [rf_umpire_id] is not null
    union
    select
        game_id, 
        v_name, 
        v_manager_id, 
        "MM"
    from
        game_log
    where
        v_manager_id is not null
    union
    select
        game_id, 
        h_name, 
        h_manager_id, 
        "MM"
    from
        game_log
    where
        h_manager_id is not null
    union
    select
        game_id, 
        case
            when h_score > v_score then h_name
            else v_name
            end, 
        winning_pitcher_id, 
        "AWP"
    from
        game_log
    where
        winning_pitcher_id is not null
    union
    select
        game_id, 
        case
            when h_score < v_score then h_name
            else v_name
            end, 
        losing_pitcher_id, 
        "ALP"
    from
        game_log
    where
        losing_pitcher_id is not null
    union
    select
        game_id, 
        case
            when
                h_score > v_score then h_name
            else v_name
            end, 
        saving_pitcher_id, 
        "ASP"
    from
        game_log
    where
        saving_pitcher_id is not null
    union
    select
        game_id, 
        case
            when h_score > v_score then h_name
            else v_name
            end, 
        winning_rbi_batter_id, 
        "AWB"
    from
        game_log
    where
        winning_rbi_batter_id is not null
    union
    select
        game_id, 
        v_name, 
        v_starting_pitcher_id, 
        "PSP"
    from
        game_log
    where
        v_starting_pitcher_id is not null
    union
    select
        game_id, 
        h_name, 
        h_starting_pitcher_id, 
        "PSP"
    from
        game_log
    where
        h_starting_pitcher_id is not null
    '''
template='''
    insert into personal_performance
    (
    game_id, 
    team_id, 
    person_id, 
    position_id
    )
    select
        game_id, 
        {hv}_name, 
        {hv}_player_{num}_id, 
        "O{num}"
    from
        game_log
    where
        {hv}_player_{num}_id is not null
    union
    select
        game_id, 
        {hv}_name, 
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos as INT)
    from
        game_log
    where
        {hv}_player_{num}_id IS NOT NULL;
    '''
run_command(c_create_personal_performance_table)
run_command(c_insert_personal_performance_table)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [139]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_personal_performance FROM personal_performance"))
q='''
    select
        personal_performance.*, 
        position.name, 
        position.category
    from
        personal_performance
    inner join
        position 
    on
        position.position_id = personal_performance.position_id
    where
        personal_performance.game_id=
            (
            select
                max(game_id)
            from
                personal_performance
            )
    order by team_id, position_id
    '''
run_query(q)

   games_game
0      171907
   games_personal_performance
0                      171907


,performance_id,person_id,team_id,game_id,position_id,name,category
0,1646114,crawa901,None,WSU188410190,UHP,Home Plate,umpire
1,3292232,crawa901,None,WSU188410190,UHP,Home Plate,umpire
2,4938350,crawa901,None,WSU188410190,UHP,Home Plate,umpire
3,6584468,crawa901,None,WSU188410190,UHP,Home Plate,umpire
4,8230586,crawa901,None,WSU188410190,UHP,Home Plate,umpire
5,9876704,crawa901,None,WSU188410190,UHP,Home Plate,umpire
6,11522822,crawa901,None,WSU188410190,UHP,Home Plate,umpire
7,13168940,crawa901,None,WSU188410190,UHP,Home Plate,umpire
8,14815058,crawa901,None,WSU188410190,UHP,Home Plate,umpire
9,16461176,crawa901,None,WSU188410190,UHP,Home Plate,umpire


In [140]:
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,position,table
5,team,table
6,game,table
7,team_performance,table
8,personal_performance,table
9,person_codes,table


In [141]:
drop_tables=[
    "game_log",
    "park_codes",
    "team_codes",
    "person_codes"
]

for tables in tables:
    run_command("drop table {}".format(tables))

show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,position,table
5,team,table
6,game,table
7,team_performance,table
8,personal_performance,table
